In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [6]:
import inspect

_ = """
to yield: to produce or to give way, both senses apply in Python when we use the `yield` keyword in a generator
A line such as yield item produces a value that is received by the caller of next(…)
it also gives way, suspending the execution of the generator so that the caller may proceed
    until it’s ready to consume another value by invoking next() again

A coroutine is syntactically like a generator: just a function with the yield keyword in
its body. However, in a coroutine, yield usually appears on the right side of an expression
(e.g., datum = yield), and it may or may not produce a value—if there is no
expression after the yield keyword, the generator yields None
The coroutine may receive data from the caller, which uses .send(datum) instead of next(…) to feed the
coroutine. Usually, the caller pushes values into the coroutine.
It is even possible that no data goes in or out through the yield keyword. Regardless of
the flow of data, yield is a control flow device that can be used to implement cooperative
multitasking: each coroutine yields control to a central scheduler so that other coroutines
can be activated.
When you start thinking of yield primarily in terms of control flow, you have the
mindset to understand coroutines.
.send(), .throw(), .close()
"""
# basic behavior of a generator used as a coroutine
def simple_coroutine(): #
    print('-> coroutine started')
    x = yield
    print('-> coroutine received:', x)


my_coro = simple_coroutine()
my_coro
next(my_coro)   # call next() to start the generator so is now waiting in a yield so we can send data
inspect.getgeneratorstate(my_coro)
my_coro.send(42)    # yield in the coroutine body evaluate to 42; now the coroutine resumes and runs until the next yield or termination
# StopIteration: control flows off the end of the coroutine body, which prompts the generator machinery to raise StopIteration
_ = """
four states of a coroutine
'GEN_CREATED'
    Waiting to start execution.
'GEN_RUNNING'
    Currently being executed by the interpreter
'GEN_SUSPENDED'
    Currently suspended at a yield expression.
'GEN_CLOSED'
    Execution has completed.

the initial call next() is often described as "priming" the coroutine
"""


<generator object simple_coroutine at 0x000001D96F1D4C50>

-> coroutine started


'GEN_SUSPENDED'

-> coroutine received: 42


StopIteration: 

In [11]:
def simple_coro2(a):
    print('-> Started: a =', a)
    b = yield a
    print('-> Received: b =', b)
    c = yield a + b
    print('-> Received: c =', c)


my_coro2 = simple_coro2(14)
from inspect import getgeneratorstate
getgeneratorstate(my_coro2)
next(my_coro2)
getgeneratorstate(my_coro2)
my_coro2.send(28)
my_coro2.send(99)
_ = """
1. next(my_coro2) prints first message and runs to yield a, yielding number 14.
2. my_coro2.send(28) assigns 28 to b, prints second message, and runs to yield a +
   b, yielding number 42.
3. my_coro2.send(99) assigns 99 to c, prints third message, and the coroutine terminates.
"""

'GEN_CREATED'

-> Started: a = 14


14

'GEN_SUSPENDED'

-> Received: b = 28


42

-> Received: c = 99


StopIteration: 

In [13]:
def averager():
    total = 0.0
    count = 0
    average = None
    while True:
        term = yield average
        total += term
        count += 1
        average = total/count


coro_avg = averager()
next(coro_avg)
coro_avg.send(10)
coro_avg.send(30)
coro_avg.send(5)
# term get value and then yield average, suspended to wait another .send()

10.0

20.0

15.0

In [16]:
# decorators for coroutine priming
_ = """
always remember to call next(co) before co.send(x)
a decorator can help
"""
from functools import wraps

def coroutine(func):
    @wraps(func)
    def primer(*args, **kwargs):
        gen = func(*args, **kwargs)
        next(gen)
        return gen
    return primer

@coroutine
def averager():
    total = 0.0
    count = 0
    average = None
    while True:
        term = yield average
        total += term
        count += 1
        average = total/count

coro_avg2 = averager()
coro_avg2.send(10)

10.0